# Altair Practice Lab

In this assignment we will be crafting a series of visualizations using
Altair to get practice working with real data in this context.

Your responses should be within the functions given,
using appropriate helper functions to help with clarity
and reduce redundancy.

## Rubric

The criteria to receive an S on this assignment is a good-faith attempt at each portion.

A good-faith attempt should either:

- be fundamentally correct producing the expected output with minimal deviations,
- **OR** contain an explanation of what does not work and _details on what was tried_.

**To receive an N, at least half of the assignment should have good-faith attempts.**

Your charts do not need to match the examples exactly!  They are helpful to get a sense of what you're after, but focus on the problem description.

## Introducing the Dataset

In the data directory you'll find three files:

**legislators.csv** which consists of ~7400 records representing state legislators, it has the following fields:

- name
- given_name
- family_name
- party: As reported by the state.
- gender: Male / Female / Other\*
- jurisdiction: This field contains an identifier for the state or jurisdiction, see below for details.
- district: the name of the district represented
- type: upper | lower - The classification of the legislative. Most states have both, but DC and NE only have an upper chamber.

Note: Accurate data on gender is hard to come by in many states. There may be irregularities in this field. This is also why this field does not make further distinctions beyond Male/Female/Other.

**populations.csv** 

A CSV file with a row with each state's population.  

**actions.csv**

A CSV file with legislative actions for a few states across a range of years:

- description: action text as reported by state
- date: date of action recorded
- classification: not used for this assignment
- state: 2-letter state postal code (note: these are upper case, other files use lower case)
- session: an identifier representing which session the action comes from (not used)

In [3]:
# PUT YOUR IMPORTS HERE
import altair as alt
alt.data_transformers.enable("vegafusion")
import polars as pl
import pandas as pd
from pathlib import Path

## Part 1: Data Exploration

First, we'll build a few exploratory visualizations to get a sense of the data for this assignment.

### 1.0: Cleaning

As mentioned above, there is no 'state' field. Jurisdiction is in the format:

`ocd-jurisdiction/country:us/state:nc/government`

and for non-states:

- `ocd-jurisdiction/country:us/district:dc/government`
- `ocd-jurisdiction/country:us/territory:pr/government`

So for our purposes, we want to add a `state` column from the two letter code after either "state:", "district:", or "territory:".
(We will treat DC and PR as states.)

Complete the function `legislators_df` which should return the data from `legislators.csv` in a dataframe, with an additional `state` column.

In [23]:
def legislators_df():
    df = pd.read_csv("data/legislators.csv")
    df['state'] = df['jurisdiction'].str.extract(r'/(state|district|territory):(\w{2})')[1].str.upper()
    return df

# render dataframe
legislators_df()

,name,given_name,family_name,gender,party,jurisdiction,district,type,state
0,Robert White,Robert C.,White,Male,Independent,ocd-jurisdiction/country:us/district:dc/govern...,At-Large,legislature,DC
1,Charles Allen,Charles,Allen,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 6,legislature,DC
2,Vince Gray,Vincent Condol,Gray,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 7,legislature,DC
3,Phil Mendelson,Philip Heath,Mendelson,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Chairman,legislature,DC
4,Trayon White,Trayon,White,Male,Democratic,ocd-jurisdiction/country:us/district:dc/govern...,Ward 8,legislature,DC
...,...,...,...,...,...,...,...,...,...
7430,Sol Higgins,Sol Yamiz,Higgins Cuadrado,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,35,lower,PR
7431,Migdalia González,Migdalia I.,Gonzalez Arroyo,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,4,upper,PR
7432,Keren Riquelme,Keren L.,Riquelme Cabrera,Female,Partido Nuevo Progresista,ocd-jurisdiction/country:us/territory:pr/gover...,At-Large,upper,PR
7433,Jocelyne Rodríguez Negrón,Jocelyne M.,Rodriguez Negron,Female,Partido Popular Democrático,ocd-jurisdiction/country:us/territory:pr/gover...,19,lower,PR


### 1.1: Initial Plot

First let's build a visualization of gender breakdowns in state legislatures.
Use the following:

- stacked bars per state
- each segment of stacked bar is gender

Your graph should somewhat resemble *imgs/ex1.1.png*.

In [43]:
def states_by_gender_initial(df):
    chart = alt.Chart(df)
    
    bar_chart = chart.mark_bar().encode(
    alt.Y('count():Q').title('Count'),
    alt.X('state:N').title('State'),
    alt.Color('gender:N').title('Gender'),
).properties(
        title='Legislators Gender'
    )
    
    return bar_chart

##should I put titles? -- yes 

# render chart
states_by_gender_initial(legislators_df())

alt.Chart(...)

### 1.2: Improvements

While it is clear from the first chart that there are more elected officials that are men than women, it is hard to compare across states.

Make the following adjustments:

- Normalize the chart so that each bar is a percentage, allowing for direct comparison across states.
- Since this is US political data, the colors red and blue have a strong meaning, associated with the Republican and Democratic parties. Change the color scheme to avoid red and blue. (I chose #8624f5 for women and #1fc3aa for men based on this article: <https://blog.datawrapper.de/gendercolor/>)
- Two states are very close to 50%, add a line at 50% using a layered chart to make it easier to see if they exceed 50% or not.

Your graph should somewhat resemble *imgs/ex1.2.png*.

In [44]:
def states_by_gender_initial(df):
    chart = alt.Chart(df)

    bar_chart = chart.mark_bar().encode(
    alt.Y('count()').stack('normalize').axis(format='%').title('Percentage Amount'), 
    alt.X('state:N').title('State'),
    alt.Color('gender:N').scale(domain=['Female','Male'], range=['#8624f5', '#1fc3aa']).title('Gender')
).properties(
        title='Legislators Gender'
    )
    #reference:https://github.com/vega/altair/issues/2059
    line = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(color='grey').encode(
        alt.Y('y:Q', title=''))

    return bar_chart + line


# render chart
states_by_gender_initial(legislators_df())

alt.LayerChart(...)

## Part 2: Party Breakdown

We'll now take a look at party control. We can start with essentially the same chart.

### 2.0 - Party Control

Copy your code from 1.2 above & modify it to use party instead of gender.  Your graph will wind up with too many parties, see `imgs/ex2.0.png`.

In [45]:
def party_control_raw(df):
    chart = alt.Chart(df)

    bar_chart = chart.mark_bar().encode(
    alt.Y('count()').stack('normalize').axis(format='%').title('Percentage'), 
    alt.X('state:N').title('State'),
    alt.Color('party:N').title('Party')
    ).properties(
        title='Legislators Party'
    )
    
    line = alt.Chart(pd.DataFrame({'y': [0.5]})).mark_rule(color='grey').encode(
        alt.Y('y:Q', title=''))

    return bar_chart + line

party_control_raw(legislators_df())

alt.LayerChart(...)

### 2.1 - Cleaning Data

The above graph still has some shortcomings:

- Most states have an upper and lower chamber, and party control may vary between them. We'll need to make two bars per state (which we'll tackle in 2.2).
- Also, there are too many variations of party as you can see here:

Let's transform the data again, adding a new column "party_code" with the following rules:

- if the word 'Democratic' appears, set party_code to 'D'
- if the word 'Republican' appears, set the party_code to 'R'
- otherwise, set the party_code to 'O'

Party data in NE, DC, and PR does not work with this scheme.
For simplicity, we will exclude them from our analysis.

For this portion, implement `clean_party_df` which should return a modified legislators DataFrame with the `party_code` column, and the rows for states 'DC', 'NE' and 'PR' dropped.

In [46]:
def clean_party_df():
    def legislators_df():
        df = pd.read_csv("data/legislators.csv")
        df['state'] = df['jurisdiction'].str.extract(r'/(state|district|territory):(\w{2})')[1].str.upper()
        df = df.drop('jurisdiction', axis='columns')
        return df
    df = legislators_df()
    #Cite: https://stackoverflow.com/questions/69424104/how-to-delete-specific-values-from-a-column-in-a-dataset-python
    df = df.drop(df[df['state'].isin(['DC', 'NE', 'PR'])].index)
    df['party_code'] = df['party'].str.extract(r'(Democratic|Republican)', expand=False).map({'Democratic': 'D', 'Republican': 'R'}).fillna('O') 
    return df

clean_party_df()

,name,given_name,family_name,gender,party,district,type,state,party_code
13,Kevin McCabe,Kevin J.,McCabe,Male,Republican,30,lower,AK,R
14,C.J. McCormick,Conrad J.,McCormick,Male,Democratic,38,lower,AK,D
15,Bill Wielechowski,Bill P.,Wielechowski,Male,Democratic,K,upper,AK,D
16,Justin Ruffridge,Justin,Ruffridge,Male,Republican,7,lower,AK,R
17,Will Stapp,Will,Stapp,Male,Republican,32,lower,AK,R
...,...,...,...,...,...,...,...,...,...
7353,Karlee Provenza,Karlee R.,Provenza,Female,Democratic,45,lower,WY,D
7354,Affie Ellis,Affie Burnside,Ellis,Female,Republican,8,upper,WY,R
7355,Abby Angelos,Abby,Angelos,Female,Republican,3,lower,WY,R
7356,Pepper Ottman,Pepper L.,Ottman,Female,Republican,34,lower,WY,R


### 2.2 - Faceted Plot

Add a function `party_control_by_chamber` that contains the following elements:

- One bar per state, **along the Y axis**.
- Each bar should consist of a stack: a blue portion, a green portion, and a red portion, corresponding to the D, O, and R `party_code` respectively.
- A vertical line at the 50% mark, indicating (likely) party control.
- Finally, facet the chart on `type` so that you get a set of bars for the lower and upper chambers.

See `imgs/ex2.2.png` for an example.

In [55]:
def party_control_by_chamber(df):
    charts = alt.Chart(df)
    bar_chart = charts.mark_bar().encode(
        alt.Y('state:N').title('State'),
        alt.X('count()', stack='normalize').axis(format='%'),
        alt.Color('party_code:N').scale(domain=['D', 'O', 'R'], range=['blue', 'green', 'red']).title('Party')
    )

    #Cite: https://github.com/vega/altair/issues/1124
    #Note: Due to merging the line and bar chart both x axis titles were put together. I tried to find the best way to
    #change this but did not find any solutions -- I will arrange a time to go to office hours to further discuss this
    #because Friday was too small of time. Therefore the best solution was to add a title that would corralate to the x-axis (i.e. percentage)
    vertline = charts.transform_calculate(
        Percentage="0.5"  
    ).mark_rule().encode(
        x='Percentage:Q'
    )
    
    final_charts = alt.layer(
        bar_chart, vertline
    ).facet(
        row='type:N'
    ).resolve_scale(
        x='independent').properties(
        title='Legislators Party'
    )
    
    return final_charts

party_control_by_chamber(clean_party_df())


alt.FacetChart(...)

## Part 3: Comparing by Population

For part three, we are interested in the relationship of various properties of legislatures to the total population of the state.

To do this, we'll need to create a combined DataFrame that mixes in data from `populations.csv`.

### 3.0 - Create Combined DataFrame

Write the function `population_combined_df`, which should return a DataFrame with the columns:

- state: abbreviation of state
- upper: total seats in upper chamber
- lower: total seats in lower chamber
- pop_2020: the 2020 population, obtained from merging with `population.csv`

**Data Note:** These numbers are based on the non-vacant seats as-of a particular day in September 2024. Vacancies will cause the counts to be off by a bit, but the general size of the legislature should be roughly the same.

In [51]:
def population_combined_df():
    add = pd.read_csv("data/populations.csv")
    add['state'] = add['state'].str.upper()
    add['population'] = add['pop_2020']

    #Note: this data has still excluded PR, DC, and NE from above in Part 2 
    df = clean_party_df()

    counts = df.groupby(["state", "type"]).size().unstack()
    
    combined_df = add.merge(counts, on="state")
    
    result_df = combined_df[['state', 'upper', 'lower','population']]
    
    return result_df

population_combined_df()

,state,upper,lower,population
0,AL,35,104,5024294
1,AK,20,40,733374
2,AZ,30,59,7157902
3,AR,35,98,3011490
4,CA,40,79,39538212
5,CO,35,65,5773707
6,CT,36,151,3605912
7,DE,21,41,989946
8,FL,40,119,21538216
9,GA,56,180,10713771


### 3.1 - Create Population vs. Seats Scatterplot

Create a new plot with two layers:

- Population on the X axis
- Number of seats on the Y axis
- Upper chamber points should be purple and use the 'triangle-up' shape.
- Lower chamber points should be orange and use the 'triangle-down' shape.
- Make a customization or two to your chart's default labels and axes, whatever you feel is appropriate.

Hint: You can layer two charts for this.

In [56]:
def scatter_pop_size():
    df = population_combined_df()
    #df['total_seats'] = df['upper'] + df['lower']

    sen = df[['state', 'population', 'upper']].rename(columns={'upper': 'seats'})
    sen['Chamber'] = 'Senate'
    house = df[['state', 'population', 'lower']].rename(columns={'lower': 'seats'})
    house['Chamber'] = 'House of Representatives'
    df_combined = pd.concat([sen, house])
    
    
    scatterplot = alt.Chart(df_combined).mark_point(size=60).encode(
        alt.X('population').title('Population'),
        alt.Y('seats').title('Count of Seats'),
        alt.Shape('Chamber', scale=alt.Scale(domain=['Senate', 'House of Representatives'], range=['triangle-up', 'triangle-down'])),
        alt.Color('Chamber', scale=alt.Scale(domain=['Senate', 'House of Representatives'], range=['purple', 'orange']))
    ).properties(
        title='Seats vs Population in States'
    )
    return scatterplot

scatter_pop_size()

alt.Chart(...)

### 3.2 - Regressions

Add two more layers, a purple & orange regression line for each chamber.  See `imgs/ex3.2.png`

Hint: See `transform_regression`.

In [58]:
def scatter_pop_size_regression():
    df = population_combined_df()
    #df['total_seats'] = df['upper'] + df['lower']

    sen = df[['state', 'population', 'upper']].rename(columns={'upper': 'seats'})
    sen['Chamber'] = 'Senate'
    house = df[['state', 'population', 'lower']].rename(columns={'lower': 'seats'})
    house['Chamber'] = 'House of Representatives'
    df_combined = pd.concat([sen, house])
    
    
    scatterplot = alt.Chart(df_combined).mark_point(size=60).encode(
        alt.X('population').title('Population'),
        alt.Y('seats').title('Count of Seats'),
        alt.Shape('Chamber', scale=alt.Scale(domain=['Senate', 'House of Representatives'], range=['triangle-up', 'triangle-down'])),
        alt.Color('Chamber', scale=alt.Scale(domain=['Senate', 'House of Representatives'], range=['purple', 'orange']))
    ).properties(
        title='Seats vs Population in States'
    )

    regression_line = scatterplot.transform_regression('population', 'seats', groupby=['chamber']
        ).mark_line()

    
    return scatterplot + regression_line



scatter_pop_size_regression()

alt.LayerChart(...)

## Part 4: Actions Heatmap

The file `actions_il-in-mi-wi_2021-2024.csv` contains nearly half a million records, representing every official action taken on a piece of legislation in these four midwestern states over the past two sessions.

Legislatures work quite differently, some meet all year, while others meet for very short periods.
By creating a heatmap of what days actions take place, we can get a sense of how different states compare.

### 4.0 - Load Actions

Complete `actions_df`, which should load the data from `actions_il-in-mi-wi_2021-2024.csv`.

Tips: 
- Make sure that the `date` column is loaded as a date type!
- Dates are in YYYY-MM-DD format, though some also have additional characters for time, which you will want to ignore.

In [5]:
def actions_df():
    df = pd.read_csv("data/actions_il-in-mi-wi_2021-2024.csv")
    df['date'] = df['date'].astype(str).str[:10]
    #Cite: #https://saturncloud.io/blog/converting-a-column-to-date-format-in-pandas-dataframe/#:~:text=Method%201%3A%20Using%20the%20to_datetime,to%20a%20standard%20datetime%20format.
    df["date"] = pd.to_datetime(df["date"]) 
    return df
actions_df()

,description,date,classification,state,session
0,"Filed with the Clerk by Rep. Marcus C. Evans, Jr.",2021-02-10,['filing'],IL,102nd
1,Filed with Secretary,2021-08-26,['filing'],IL,102nd
2,Filed with Secretary by Sen. Jacqueline Y. Col...,2021-02-25,['filing'],IL,102nd
3,Filed with Secretary,2022-01-19,['filing'],IL,102nd
4,Filed with Secretary,2022-11-29,['filing'],IL,102nd
...,...,...,...,...,...
456400,Partial veto failed to pass notwithstanding th...,2022-05-17,[],WI,2021
456401,Fiscal estimate received,2021-02-08,['receipt'],WI,2021
456402,Fiscal estimate received,2021-02-10,['receipt'],WI,2021
456403,Placed on calendar 5-17-2022 pursuant to Joint...,2022-05-17,[],WI,2021


### 4.1 - Actions Heatmap

Generate a heatmap (using `mark_rect`) with:

- a row per state
- each row consists of shaded marks with shading based on the total action count for a given week

Tip: Use the 'yearweek(date)' aggregation for the X channel.

See `imgs/ex4.1.png`.

In [70]:
def actions_heatmap_scaled(df):
    #Cite:https://stackoverflow.com/questions/45281297/group-by-week-in-pandas
    data = (df.groupby(['state', pd.Grouper(key='date', freq='W')])
            .size()
            .reset_index(name='count'))

    heatmap = alt.Chart(data).mark_rect().encode(
        alt.X('yearweek(date):O').title('Date (year-week)'),
        alt.Y("state:N").title("State"),
        alt.Color('count:Q').title('Actions Count'),
    ).properties(width=800, height=100,title='Actions Heatmap')
    
    return heatmap

actions_heatmap_scaled(actions_df())

alt.Chart(...)

### 4.2 - Excluding IL Outliers

Illinois clearly dominates the above graph, below modify two calls to `actions_heatmap` with a modified dataframe that excludes IL, and a modified dataframe that only includes IL.

(Note how by using functions in our dataframe we can more easily reuse portions by making small adjustments to the data.)

See `ex4.2a.png` and `ex4.2b.png`

In [71]:
df = actions_df()
actions_heatmap_scaled(df[df['state'] != 'IL'])

alt.Chart(...)

In [72]:
df = actions_df()
actions_heatmap_scaled(df[df['state'] == 'IL'])

alt.Chart(...)

#### 4.3 - Cumulative Line Chart

Another way to view this data would be with a cumulative line chart.

Create a chart with:

- days on the X axis
- cumulative actions to date on the Y axis
- one line per state

Hint: To do this you will need to look at the `transform_window` function.

See `ex4.3.png` for an example.

In [7]:
def actions_cumulative(df):
    data = (df.groupby(['state', pd.Grouper(key='date', freq='W')])
            .size()
            .reset_index(name='count'))
    
    chart = alt.Chart(data).mark_line().transform_window(
        sort=[{'field': 'date'}],
        frame=[None, 0],
        cumulative_actions='sum(count)',
        groupby=['state']
    ).encode(
        alt.X('date:T').title('Date'),
        alt.Y('cumulative_actions:Q').title('Cumalative Actions'),
        alt.Color('state:N').title('State')
    ).properties(width=500, height=100,title='Cumalative Actions Line Chart')
    
    return chart

actions_cumulative(actions_df())

alt.Chart(...)